# Web Scraping for Used Car Prices from Autotrader.ca

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing required modules
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import requests
import re
import pymongo
import pandas as pd
from sqlalchemy import create_engine,inspect, func
import numpy as np
import time
#from config import api_key

In [27]:
# This is to initialize Splinter for Windows users
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [28]:
# list to store extracted values

title_list=[]
year_list=[]
price_list=[]
loc_list=[]
date_list=[]
milage_list=[]
car_list=[]
model_list=[]
trans_list=[]

# loop to scrap website
for item_num in range(100, 50001, 100):
    url = "https://www.autotrader.ca/cars/?rcp=100&rcs="+str(item_num)+"&srt=9&prx=-1&hprc=True&wcp=True&sts=New-Used&inMarket=advancedSearch"
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    results = soup.find_all('div', class_='newSRPDesign')




#i=1

#while i<=4:
    
    for result in results:
        #url = "https://www.autotrader.ca/cars/?rcp=15&rcs=0&srt=9&prx=-1&hprc=True&wcp=True&sts=New-Used&inMarket=advancedSearch"
        #browser.visit(url)
        #html = browser.html
        #soup = BeautifulSoup(html, 'html.parser')
        #results = soup.find_all('div', class_='newSRPDesign')
        
        try:
            
            
            # scrape the price
            price = result.find('span', class_='price-amount').text
            price_list.append(price)
            # scrape the  Title
            ctitle = result.find('div', class_='listing-details').h2.a.span.text
            #words=['lease', 'take over', 'finance', 'financing', 'leasing', 'takeover', 'financed', 'leased', 'transfer', 'transferring']
            #for word in words:
            
                #if word in result.find('div', class_='title').text.split():
                    #print('a listing including lease or finance was removed')
                #elif word in result.find('div', class_='description').text.split():
                        #print('a listing including lease or finance was removed')
                #else:
            btitle= ctitle.split('\n')[1].replace('                                    ','')
            title = btitle.split('                                    ')[0]
            year = title.split(" ")[0]
            cartype= title.split(" ")[1]
            model = title.split(" ")[2]
            title_list.append(title)
            year_list.append(year)
            car_list.append(cartype)
            model_list.append(model)
         
            
            
        # scrape the  Location
            #location = result.find('div', class_='location').text.split()[0]
            #loc_list.append(location)
        
         
        
        
        
        # scrape the  Date
            #date = result.find('span', class_='date-posted').text
            #date_list.append(date)
        # scrape the  details
            details = result.find('div', class_='kms').text#.split('"')[0]         
            milage=details
           
            
            milage_list.append(milage)            
   
    
        except AttributeError:
            date=''
        except IndexError:
            cartype=''
            model=''
            
#browser.find_link_by_text(i+1).first.click()
#i=i+1


# store the extracted valuse in dictionary            
Dict={}
Dict["Title"]=title_list
Dict["Car Type"]=car_list
Dict["Model"]=model_list
Dict["Price"]=price_list
#Dict["Location"]=loc_list
#Dict["Date"]=date_list
Dict["Milage"]=milage_list
Dict["Year"]=year_list
#Dict["Transmission"]=trans_list       

In [29]:
df=pd.DataFrame.from_dict(Dict, orient='index')
df=df.transpose()
df

,Title,Car Type,Model,Price,Milage,Year
0,2017 Ford Escape S AIR CLIMATISÉ - Caméra de R...,Ford,Escape,"$14,494","\nMileage 70,742 km\n ...",2017
1,2017 Volkswagen Jetta Sedan Wolfsburg Edition,Volkswagen,Jetta,"$18,900","\nMileage 55,496 km\n ...",2017
2,2014 Audi A4 Progressiv quattro SLINE/NAVI/B.C...,Audi,A4,"$17,995","\nMileage 85,000 km\n ...",2014
3,2020 Chevrolet SILVERADO 3500HD High Country,Chevrolet,SILVERADO,"$95,406","\nMileage 97,531 km\n ...",2020
4,2016 BMW 428i xDrive Gran Coupe SAFETY INSPECTED.,BMW,428i,"$18,517","\nMileage 74,625 km\n ...",2016
5,"2016 Ram 1500 SLT CREW CAB, BLUETOOTH, HEMI, 4X4",Ram,1500,"$29,999","\nMileage 20,762 km\n ...",2016
6,2017 Jeep Cherokee 4WD 4dr Sport,Jeep,Cherokee,"$24,888","\nMileage 179,985 km\n ...",2017
7,2015 Chevrolet Traverse AWD 4dr LS + ACCIDENT ...,Chevrolet,Traverse,"$10,989","\nMileage 16,684 km\n ...",2015
8,2019 Nissan Kicks SV|Sat.Radio|Backup_Cam|Heat...,Nissan,Kicks,"$18,836","\nMileage 6,890 km\n ...",2019
9,2019 Audi Q7 Progressiv w/Tow & Driver Assist ...,Audi,Q7,"$69,995","\nMileage 70,263 km\n ...",2019


In [30]:
df['Title'].value_counts()

2020 Ford F-150 XLT                                                                   257
2020 Ram 1500 Classic Express                                                         178
2020 Ford F-150                                                                       148
2020 Toyota Corolla                                                                   142
2020 GMC Sierra 1500                                                                  131
2020 Toyota Corolla LE                                                                129
2020 Toyota RAV4                                                                      109
2020 Toyota RAV4 AWD LE                                                               107
2020 Hyundai Elantra PREFERRED W/SUN & SAFETY PACKAGE                                  90
2020 Toyota Corolla LE CVT                                                             83
2020 Ram 1500 Classic                                                                  82
2020 Ford 

In [40]:
df["Price"] = df["Price"].str.replace("$", "")
df["Price"] = df["Price"].str.replace(",", "")
df["Milage"] = df["Milage"].str.replace(",", "")
df["Milage"] = df["Milage"].str.replace("\nMileage", "")
df["Milage"] = df["Milage"].str.replace("\n", "")
df["Milage"] = df["Milage"].str.replace("km", "")

In [41]:
df.reset_index()
df.dropna()
df.drop_duplicates()
df = df.mask(df.eq('None')).dropna()

In [42]:
df

,Title,Car Type,Model,Price,Milage,Year
0,2017 Ford Escape S AIR CLIMATISÉ - Caméra de R...,Ford,Escape,14494,70742,2017
1,2017 Volkswagen Jetta Sedan Wolfsburg Edition,Volkswagen,Jetta,18900,55496,2017
2,2014 Audi A4 Progressiv quattro SLINE/NAVI/B.C...,Audi,A4,17995,85000,2014
3,2020 Chevrolet SILVERADO 3500HD High Country,Chevrolet,SILVERADO,95406,97531,2020
4,2016 BMW 428i xDrive Gran Coupe SAFETY INSPECTED.,BMW,428i,18517,74625,2016
5,"2016 Ram 1500 SLT CREW CAB, BLUETOOTH, HEMI, 4X4",Ram,1500,29999,20762,2016
6,2017 Jeep Cherokee 4WD 4dr Sport,Jeep,Cherokee,24888,179985,2017
7,2015 Chevrolet Traverse AWD 4dr LS + ACCIDENT ...,Chevrolet,Traverse,10989,16684,2015
8,2019 Nissan Kicks SV|Sat.Radio|Backup_Cam|Heat...,Nissan,Kicks,18836,6890,2019
9,2019 Audi Q7 Progressiv w/Tow & Driver Assist ...,Audi,Q7,69995,70263,2019


In [43]:
from config import password
connection_string = "postgres:"+password+"@localhost:5432/hondadb"
engine = create_engine(f'postgresql://{connection_string}')

In [44]:
# Confirm tables
#engine.table_names()
inspector = inspect(engine)
inspector.get_table_names()

['cleanedcardb2', 'cleanedcardb', 'cleanedhondadb2', 'hondadb', 'cardb']

In [45]:
inspector.get_columns("cardb")

[{'name': 'index',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'Title',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'Car Type',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'Model',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'Price',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'Milage',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'Year',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None}]

In [46]:
df.to_sql(name='cardb', con=engine, if_exists='replace', index=True)

In [47]:
sql_DF = pd.read_sql("SELECT * FROM cardb",
                     con=engine)

In [48]:
sql_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38871 entries, 0 to 38870
Data columns (total 7 columns):
index       38871 non-null int64
Title       38871 non-null object
Car Type    38871 non-null object
Model       38871 non-null object
Price       38871 non-null object
Milage      38871 non-null object
Year        38871 non-null object
dtypes: int64(1), object(6)
memory usage: 2.1+ MB
